In [1]:
import ee
import geemap
ee.Authenticate()

ModuleNotFoundError: No module named 'ee'

In [2]:
ee.Initialize()

In [3]:
save = False

startDate = '2020-01-01' #// window size (in months) for the Sentinel-1 composite
endDate = '2020-12-31' #// last day of the composite

SENMAU2 = ee.Geometry.Polygon([
  [-17.557569245407162,14.521147928653363],
  [-16.637464264938412,14.521147928653363],
  [-16.637464264938412,14.934208283183736],
  [-17.557569245407162,14.934208283183736],
  [-17.557569245407162,14.521147928653363]])

Area = "SENMAU2"

if Area=="SENMAU2":
    exportArea = SENMAU2
else:
    # other Areas
    pass

## Open Buildings


In [4]:
GBuildingMap = geemap.Map()
# ///////////////// Google Open Buildings //////////////////////////////////
t = ee.FeatureCollection('GOOGLE/Research/open-buildings/v1/polygons')
# //var t_060_065 = t.filter('confidence >= 0.60 && confidence < 0.65');
# //var t_065_070 = t.filter('confidence >= 0.65 && confidence < 0.70');
t_gte_070 = t.filter('confidence >= 0.70')
t_filtered = t.filterBounds(exportArea)
# print(t_filtered.size());
GBuildingMap.addLayer(t, {'color': 'FF0000'}, 'Buildings')


In [5]:

GBuildingtask = ee.batch.Export.table.toDrive(
   collection = t_filtered,
   description = 'GBuildings-GEE-'+Area,
   fileFormat= 'CSV',
   folder= "GEEdownload_bd"
)

if save:
   GBuildingtask.start()
   GBuildingtask.status()

GBuildingMap.centerObject(exportArea, 9)
GBuildingMap


Map(center=[14.728066996095398, -17.097516755172467], controls=(WidgetControl(options=['position', 'transparen…

In [6]:
GBuildingtask.status()

{'state': 'UNSUBMITTED'}

## Sentinel 2

In [12]:
def maskS2clouds(image):
    qa = image.select('QA60')

    #   // Bits 10 and 11 are clouds and cirrus, respectively.
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11

    #   // Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cirrusBitMask).eq(0))

    #   // Return the masked and scaled data, without the QA bands.
    return image.updateMask(mask).divide(10000).select("B.*").copyProperties(image, ["system:time_start"])

In [15]:
# /////////////////// Sentinel 2 - Preprocessed L2A ///////////////////////////////////////////
# // Map the function over one year of data and take the median.
# // Load Sentinel-2 TOA reflectance data.
collection = ee.ImageCollection('COPERNICUS/S2_SR').filterDate(startDate, endDate).filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)).map(maskS2clouds)
compositeS2 = collection.median()


# // Map.addLayer(compositeS2);
task = ee.batch.Export.image.toDrive(
   image= compositeS2,
   description= 'Sentinel-2L2A-'+Area,
   scale= 10,
   region= exportArea,
   maxPixels=80000000000,
   folder="GEEdownload_bd_L2A"
)
if save:
   task.start()
   task.status()


visualization = { 'min': 0.00, 'max': 0.25,  'bands': ['B4', 'B3', 'B2'], }
S2Map = geemap.Map()
S2Map.addLayer(compositeS2, visualization, 'RGB_L2A')
S2Map.centerObject(exportArea, 9)
S2Map

Map(bottom=754.0, center=[14.728066996095398, -17.097516755172467], controls=(WidgetControl(options=['position…

In [ ]:
task.status()

## Sentinel 1


In [ ]:

sentinel1 = ee.ImageCollection('COPERNICUS/S1_GRD').filterDate(startDate, endDate)


# // Filter the Sentinel-1 collection by metadata properties.
# vvVhIw = sentinel1
#   .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
#   .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))
#   .filter(ee.Filter.eq('instrumentMode', 'IW'))
#   .filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))
#   .filterBounds(exportArea);


# TODO!!!!!!!!!!!!
S1image = 



task = ee.batch.Export.image.toDrive(
  image= S1image,
  description= 'GSentinel-1-'+Area,
  scale= 10,
  region= exportArea,
  maxPixels=80000000000, 
  folder= "GEEdownload_bd"
)

if save:
   task.start()
   task.status()


In [ ]:

S1Map = geemap.Map()
S1Map.addLayer(S1image, {min: [-18, -26, -26], max: [4, 0, 0]}, 'composite')
S2Map.centerObject(exportArea, 9)

S1Map


In [ ]:
task.status()

## Nightlights

In [10]:

# //////////////////// VIIRS ////////////////////////////////////

dataset = ee.ImageCollection('NOAA/VIIRS/DNB/MONTHLY_V1/VCMCFG').filter(ee.Filter.date(startDate, endDate))
nighttimeLights = dataset.select('avg_rad')
nighttimeLights = nighttimeLights.filterBounds(exportArea).median()
  



taskVIIRS = ee.batch.Export.image.toDrive(
  image= nighttimeLights,
  description= 'VIIRS-1-'+Area,
  scale= 10,
  region= exportArea,
  maxPixels=80000000000, 
  folder= "GEEdownload_bd"
)



MapVIIRS = geemap.Map()
nighttimeLightsVis = { 'min': -1.5,  'max': 60.0, }
MapVIIRS.addLayer(nighttimeLights, nighttimeLightsVis, 'Nighttime Lights')  
MapVIIRS.centerObject(exportArea, 9)
MapVIIRS

Map(center=[14.728066996095398, -17.097516755172467], controls=(WidgetControl(options=['position', 'transparen…